# Mount drive and append path to PYTONPATH


In [ ]:
import os
import sys

from google.colab import drive

drive.mount("/content/drive")
sys.path.append("/content/drive/MyDrive/DeepLCMS/train_google_colab")

# Import and install libraries

In [ ]:
%%capture
!pip install lightning
!pip install timm
!pip install torchinfo
!pip install scikit-posthocs
!pip install optuna
!pip install torchcam

In [ ]:
import colab_functions
import colab_utils
import pandas as pd
import prepare_data
import timm
import train_NN
from lightning.pytorch import loggers, callbacks, tuner, trainer
from pathlib import Path
from tqdm import tqdm
from IPython.display import clear_output
import seaborn as sns
import matplotlib.pyplot as plt
import scikit_posthocs as sp

In [ ]:
# Set the CUDA_VISIBLE_DEVICES environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Unzip data

In [ ]:
%%script echo skipping
!unzip -q "*.zip"

# Check if GPU is used

In [ ]:
device = colab_functions.get_device()

# Getting a tunable model

In [ ]:
class Resnet_model_tune(pl.LightningModule):
    def __init__(self, hyperparameters):
        super().__init__()
        self.hyperparameters = hyperparameters
        self.model = create_model("resnet50d.a3_in1k", pretrained=True, num_classes=1)

        # Freeze all layers except for the last one
        for param in self.model.parameters():
            param.requires_grad = False

        self.model.fc = nn.Sequential(
            nn.Linear(in_features=2048, out_features=512, bias=True),
            nn.ReLU(),
            nn.Dropout(p=self.hyperparameters["dropout"]),
            nn.Linear(in_features=512, out_features=256, bias=True),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=1, bias=True),
        )

    def forward(self, x):
        x = self.model(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch

        loss_fn = nn.BCELoss()

        y_pred_logits = self(x).squeeze()
        y_pred = torch.sigmoid(y_pred_logits)
        loss = loss_fn(y_pred, y.float())

        self.log(
            "train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True
        )

        # Calculate metrics

        # Calculate Accuracy
        y_pred_class = torch.round(y_pred)
        acc = (y_pred_class == y).sum().item() / len(y_pred)
        self.log(
            "train_acc", acc, on_step=False, on_epoch=True, prog_bar=False, logger=True
        )
        # Calculate F1
        metric_f1 = BinaryF1Score().to(y.device)
        f1 = metric_f1(y_pred_class, y)
        self.log(
            "train_f1", f1, on_step=False, on_epoch=True, prog_bar=False, logger=True
        )
        # Calculate Precision
        metric_precision = BinaryPrecision().to(y.device)
        precision = metric_precision(y_pred_class, y)
        self.log(
            "train_precision",
            precision,
            on_step=False,
            on_epoch=True,
            prog_bar=False,
            logger=True,
        )
        # Calculate Recall
        metric_f1 = BinaryRecall().to(y.device)
        recall = metric_f1(y_pred_class, y)
        self.log(
            "train_recall",
            recall,
            on_step=False,
            on_epoch=True,
            prog_bar=False,
            logger=True,
        )

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch

        loss_fn = nn.BCELoss()

        y_pred_logits = self(x).squeeze()
        y_pred = torch.sigmoid(y_pred_logits)
        loss = loss_fn(y_pred, y.float())
        self.log(
            "val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True
        )

        # Calculate metrics

        # Calculate Accuracy
        y_pred_class = torch.round(y_pred)
        acc = (y_pred_class == y).sum().item() / len(y_pred)
        self.log(
            "val_acc", acc, on_step=False, on_epoch=True, prog_bar=True, logger=True
        )
        # Calculate F1
        metric_f1 = BinaryF1Score().to(y.device)
        f1 = metric_f1(y_pred_class, y)
        self.log("val_f1", f1, on_step=False, on_epoch=True, prog_bar=True, logger=True)

        # Calculate Precision
        metric_precision = BinaryPrecision().to(y.device)
        precision = metric_precision(y_pred_class, y)
        self.log(
            "val_precision",
            precision,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        # Calculate Recall
        metric_f1 = BinaryRecall().to(y.device)
        recall = metric_f1(y_pred_class, y)
        self.log(
            "val_recall",
            recall,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        if isinstance(batch, list):
            # Assuming the first element in the list is the input tensor
            input_tensor = batch[0]
            return self(input_tensor)
        else:
            # If batch is already a tensor, proceed as usual
            print("Input Shape:", batch.shape)
            return self(batch)

    def configure_optimizers(self):
        optimizer = None

        if self.hyperparameters["optimizer"] == "Adam":
            optimizer = Adam(
                self.parameters(), lr=self.hyperparameters["lr"], weight_decay=2e-5
            )
        elif self.hyperparameters["optimizer"] == "SGD":
            optimizer = SGD(
                self.parameters(), lr=self.hyperparameters["lr"], weight_decay=2e-5
            )
        elif self.hyperparameters["optimizer"] == "RMSprop":
            optimizer = RMSprop(
                self.parameters(), lr=self.hyperparameters["lr"], weight_decay=2e-5
            )
        else:
            raise ValueError(
                f"Unsupported optimizer: {self.hyperparameters['optimizer']}"
            )

        scheduler = None

        if self.hyperparameters["scheduler"] == "ReduceLROnPlateau":
            scheduler = {
                "scheduler": ReduceLROnPlateau(
                    optimizer, mode="min", factor=0.1, patience=3
                ),
                "interval": "epoch",
                "monitor": "val_loss",
            }
        elif self.hyperparameters["scheduler"] == "CosineAnnealingLR":
            scheduler = CosineAnnealingLR(optimizer, T_max=50, eta_min=0)

        return [optimizer], [scheduler]


def objective(trial):
    hyperparameters = {
        "optimizer": trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"]),
        "scheduler": trial.suggest_categorical(
            "scheduler", ["ReduceLROnPlateau", "CosineAnnealingLR"]
        ),
        "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
        "dropout": trial.suggest_float("dropout", 0.01, 1),
    }

    model = Resnet_model_tune(hyperparameters)
    logger = CSVLogger("logs", name=str(trial.number))
    trainer = pl.Trainer(
        logger=logger,
        max_epochs=50,
        callbacks=[
            EarlyStopping(monitor="val_loss", patience=10),
            PyTorchLightningPruningCallback(trial, monitor="val_loss"),
        ],
    )

    trainer.fit(model, train_dataloader, val_dataloader)

    return trainer.callback_metrics["val_loss"].item()


def print_callback(study, trial):
    print(
        f"Trial {trial.number} finished with value: {trial.value} and parameters: {trial.params}"
    )

In [ ]:
(
    preprocess_train,
    preprocess_val,
    preprocess_test,
) = prepare_data.get_timm_transforms(train_NN.Resnet_model())

(
    train_dataloader,
    val_dataloader,
    test_dataloader,
) = prepare_data.get_dataloaders(
    preprocess_train=preprocess_train,
    preprocess_val=preprocess_val,
    preprocess_test=preprocess_test,
)

# bug encountered:
# https://github.com/pytorch/pytorch/issues/67978
study.optimize(objective, n_trials=100, callbacks=[print_callback])

In [ ]:
with open("optuna_params.pickle", "wb") as handle:
    pickle.dump(study.best_params, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open("optuna_params.pickle", "rb") as handle:
    optuna_params = pickle.load(handle)

print(study.best_params == optuna_params)

In [ ]:
plot_parallel_coordinate(study)

In [ ]:
plot_contour(study)

In [ ]:
plot_param_importances(study)

# Evaluate results

In [ ]:
results_df = colab_functions.get_experiment_results().assign(
    experiment=lambda df: df.experiment.astype(int)
)
optuna_trials = pd.read_csv("optuna_trials.csv")

# merge results_df with optuna_trials so that we have access to the full training
# data with all epochs
# this is needed since optuna made a decision based on overfitted data

df = results_df.merge(optuna_trials, left_on="experiment", right_on="number")

Optuna originally selected trial #27, which only achieved a validation loss of 0.1353. This is because Optuna considers the validation loss of the last epoch before terminating the trial due to overfitting. Therefore, the final conclusion reached by Optuna is based on an already overfitted model. Based on the learning curves logged, we can determine the best conditions and the number of epochs we should train our model for.

These are the validation losses regarding trial #27:

In [ ]:
(df.query("experiment == 27 and variable.str.contains('val_loss')")).sort_values(
    by="epoch"
)

The absolute minimum validation loss reached during the optimization procedure was 0.1289 at epoch 3 for experiment 12. However, by the next epoch, the model was overfitted and recorded a final validation loss of 0.2212, which is higher than the validation loss of trial #27. Therefore, trial #27 was selected over experiment 12.

In [ ]:
(
    df.loc[lambda df: df.groupby("variable")["value_x"].idxmin(), :].query(
        "variable.str.contains('val_loss') "
    )
)

These experiments represent the overall best trials and corresponding epochs per metric monitored. As you can see, trial #12 is still present with a validation accuracy of 0.98. Most of these trials used the `Adam` optimizer and `CosineAnnealingLR` scheduler. Based on these findings, we can opt for the conditions described for trial #12.

In [ ]:
(
    df.loc[lambda df: df.groupby("variable")["value_x"].idxmax(), :].query(
        "~variable.str.contains('train|loss')"
    )
)